In [ ]:
#%reset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
import lightgbm as lgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## load data

In [ ]:
#hpg使わない
air_vis = pd.read_csv('../input/air_visit_data.csv')
air_res = pd.read_csv('../input/air_reserve.csv')
store = pd.read_csv('../input/air_store_info.csv')
#hpg_res = pd.read_csv('../input/hpg_reserve.csv')
#hpg_store = pd.read_csv('../input/hpg_store_info.csv')
#id_store = pd.read_csv('../input/store_id_relation.csv')
date_info = pd.read_csv('../input/date_info.csv')
samp_sub = pd.read_csv('../input/sample_submission.csv')

In [ ]:
def make_date(data):
    data['visit_date'] = pd.to_datetime(data['visit_date'])
    data['year'] = data['visit_date'].dt.year
    data['month'] = data['visit_date'].dt.month
    #data['week'] = data['visit_date'].dt.week
    data['dow'] = data['visit_date'].dt.dayofweek
    data['visit_date'] = data['visit_date'].dt.date
    return data

In [ ]:
# split_date
air_vis = pd.merge(air_vis,date_info,left_on='visit_date',right_on='calendar_date',how='left')
air_vis['sat_sun'] = air_vis['day_of_week'].apply(lambda x : 1 if (x == 'Saturday') or (x == 'Sunday') else 0)
air_vis = make_date(air_vis)

In [ ]:
# max/min/stdは全体/祝日/土日であればOK
vis_gp = air_vis.groupby('air_store_id').agg({'visitors':['max','min','mean','median']})
vis_count = air_vis.groupby('air_store_id').size().to_frame().rename(columns={0:'count'}).reset_index()
vis_gp_sat_sun = air_vis.pivot_table(index = 'air_store_id',columns='sat_sun',values='visitors',aggfunc='median')
vis_gp_holiday = air_vis[air_vis['holiday_flg'] == 1].groupby('air_store_id')['visitors'].median().to_frame()
vis_gp_mon = air_vis.pivot_table(index = 'air_store_id',columns='month',values='visitors',aggfunc='median')
vis_gp_mon = vis_gp_mon.apply(lambda x : x.fillna(x.mean(axis=0)))

vis_gp = pd.merge(vis_gp,vis_count,on='air_store_id',how='left')
vis_gp = pd.merge(vis_gp,vis_gp_sat_sun,on='air_store_id',how='left')
vis_gp = pd.merge(vis_gp,vis_gp_holiday,on='air_store_id',how='left')
vis_gp = pd.merge(vis_gp,vis_gp_mon,on='air_store_id',how='left')

vis_gp.columns = ['air_store_id','count','vis_max','vis_min','vis_mean','vis_median','vis_weekday_median','vis_sat_sun_median','vis_holiday_median'
                  ,'vis_month_1_mean','vis_month_2_mean','vis_month_3_mean','vis_month_4_mean','vis_month_5_mean','vis_month_6_mean'
                  ,'vis_month_7_mean','vis_month_8_mean','vis_month_9_mean','vis_month_10_mean','vis_month_11_mean','vis_month_12_mean']

# add：色々入れるとLB下がる。絞ることにした
# vis_gp = vis_gp[['air_store_id','count','vis_max','vis_min','vis_mean','vis_median','vis_weekday_median','vis_sat_sun_median','vis_holiday_median']]


In [ ]:
## 時間あればやる 多分精度変わると思う
# vis_sat_sun_mean と vis_holiday_meanの欠損埋めるで
# 方針：8月（長期休み）と10月（平日多め）の平均人数を比較
# 8月 > 10月の場合：平日の1.5倍と仮定
# 10月 < 8月の場合：平日の0.5倍と仮定

In [ ]:
# ジャンルと都道府県までを特徴にする
# 表記揺れの処理は本来必要だが実施しない

store['prefecture'] = store['air_area_name'].map(lambda x: x.split(' ')[0])
#store['city'] = store['air_area_name'].map(lambda x: x.split(' ')[1])
store = store[['air_store_id','air_genre_name','prefecture']]

# trans label_encoder
lbl = preprocessing.LabelEncoder()
store['air_genre_name'] = lbl.fit_transform(store['air_genre_name'])
store['prefecture'] = lbl.fit_transform(store['prefecture'])
#store['city'] = lbl.fit_transform(store['city'])


In [ ]:
#予約数 これも使えないか
air_res = air_res.drop(['visit_datetime'],axis=1).rename(columns={'reserve_datetime':'visit_date'})
air_res = make_date(air_res)

In [ ]:
air_res.head(3)

In [ ]:
air_res_mon.head(3)

In [ ]:
air_res_mon = air_res.pivot_table(index = 'air_store_id',columns='month',values='reserve_visitors',aggfunc='median')
air_res_mon = air_res_mon.apply(lambda x : x.fillna(x.mean(axis=0))).reset_index()
air_res_mon.columns = ['air_store_id','res_month_1_mean','res_month_2_mean','res_month_3_mean','res_month_4_mean','res_month_5_mean','res_month_6_mean'
                  ,'res_month_7_mean','res_month_8_mean','res_month_9_mean','res_month_10_mean','res_month_11_mean','res_month_12_mean']

In [ ]:
# merge
store_dm = pd.merge(vis_gp,store,on='air_store_id',how='left')

# make datamart

In [ ]:
# testでも同じの作る
# 5 sat
samp_sub = pd.read_csv('../input/sample_submission.csv')
samp_sub['air_store_id'] = samp_sub.id.map(lambda x: '_'.join(x.split('_')[:-1]))
samp_sub['calendar_date'] = samp_sub.id.map(lambda x: x.split('_')[2])

samp_sub = pd.merge(samp_sub,date_info,on='calendar_date',how='left')
samp_sub['sat_sun'] = samp_sub['day_of_week'].apply(lambda x : 1 if (x == 'Saturday') or (x == 'Sunday') else 0)
samp_sub = make_date(samp_sub.rename(columns={'calendar_date':'visit_date'}))

In [ ]:
# select feature
air_vis_train = air_vis[['air_store_id','visitors','sat_sun','holiday_flg','year','month','dow']]
air_vis_test = samp_sub[['air_store_id','sat_sun','holiday_flg','year','month','dow']]

In [ ]:
# join store_dm
train = pd.merge(air_vis_train,store_dm ,on = 'air_store_id',how='left')
test = pd.merge(air_vis_test,store_dm ,on = 'air_store_id',how='left')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
del train['air_store_id'],test['air_store_id']

# Modeling

In [ ]:
y = np.log1p(train['visitors'].values)
del train['visitors']

In [ ]:
# transfrom category_val
category_column = ['sat_sun','holiday_flg','year','month','dow','air_genre_name','prefecture']
train[category_column] = train[category_column].astype('category')
test[category_column] = test[category_column].astype('category')

In [ ]:
test.describe()

In [ ]:
# Create training / validation split
split = 200000
x_train, y_train, x_valid, y_valid = train[:split], y[:split], train[split:], y[split:]

print('Building DMatrix...')

lgtrain = lgb.Dataset(x_train, label=y_train)
lgval = lgb.Dataset(x_valid, label=y_valid)

params = {
    "objective" : "regression",
    "metric" : "rmse",
    "num_leaves" : 30,
    "learning_rate" : 0.05,
    "verbosity" : -1,
    'max_depth': 4,
    'use_missing': True
}

model = lgb.train(params
                  ,lgtrain
                  ,100000
                  ,valid_sets=[lgval]
                  ,early_stopping_rounds=100
                  ,verbose_eval=1000
                  #,evals_result=evals_result
                  #今回はskipしたが、sample_weightを本来つけるべき（小サンプルに重みを高くつけるべき）
                 )

pred_test_y = model.predict(test, num_iteration=model.best_iteration)

In [ ]:
### Feature Importance ###
fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
plt.show()

In [ ]:
samp_sub = pd.read_csv('../input/sample_submission.csv')
samp_sub['visitors'] = np.expm1(pred_test_y)

In [ ]:
samp_sub.head(3)

In [ ]:
samp_sub.to_csv('submission.csv', index=False)